In [47]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


Reading the dataframe we prepared in previous question from csv file into dataframe df

In [48]:
df=pd.read_csv('TorontoGeoData.csv')

In [49]:
df.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [50]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [51]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Getting geographical coordinates of Toronto

In [52]:
toronto_add='Toronto, Ontario' #CITY ADDRESS
geolocator=Nominatim(user_agent='toronto_explorer')
location=geolocator.geocode(toronto_add)
latitude=location.latitude
longitude=location.longitude
print("The coordinates of Toronto are {},{}".format(latitude,longitude))

The coordinates of Toronto are 43.6534817,-79.3839347


In [53]:
Toronto_map=folium.Map([latitude,longitude],zoom_start=10)
for lat,long,postalcode,borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['PostalCode'],df['Borough'],df['Neighborhood']):
    label='Postal Code:{},Neighborhoods:{},Borough:{}'.format(postalcode,neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat,long],radius=5,color='blue',popup=label,
                        fill=True,fill_color='#3186cc',fill_opacity=0.7,
                        parse_html=True).add_to(Toronto_map)

In [54]:
Toronto_map

In [55]:
import requests# library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize

We create a function to extract dataframe of top 100 venues within 500m radius of each of the postal codes in Toronto¶


In [56]:
limit=100
CLIENT_ID = 'P5CRNJZ5XQNB3R1QD24JMPB5WF1UU40SKBR3OS12HRC0K3X5'
CLIENT_SECRET = 'BEQ3PWLYHPDJXZOXP4LUGYC2TV4XLYYQLSRVG3PSZH14PMW5'
VERSION = 20180605
def getNearbyVenues(codes,names,latitudes,longitudes,radius=500):
    venues_list=[]
    for code,name,lat,lng in zip(codes,names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)
        results=requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(code,name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],
                          v['venue']['location']['lng'],
                          v['venue']['categories'][0]['name'])for v in results])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['PostalCode','Neighborhood','Neighborhood Latitude',
                          'Neighborhood Longitude','Venue name','Venue Latitude','Venue Longitude','Venue Category']
    return(nearby_venues)

In [57]:
Toronto_Venues=getNearbyVenues(codes=df['PostalCode'],
                               names=df['Neighborhood'],
                               latitudes=df['Latitude'],
                               longitudes=df['Longitude'],radius=500)

In [58]:
Toronto_Venues.head()

,PostalCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue name,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M3A,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [59]:
Toronto_Venues.shape

(1997, 8)

In [60]:
Toronto_Venues.groupby('PostalCode').count()[['Venue name']]

,Venue name
PostalCode,
M1B,1
M1C,2
M1E,9
M1G,4
M1H,8
...,...
M9N,2
M9P,8
M9R,3


In [61]:
print('There are {} unique venue categories'.format(len(Toronto_Venues['Venue Category'].unique())))


There are 262 unique venue categories


Now we analyze each neighborhood by creating a dataframe with one hot encoding¶


In [62]:

Toronto_onehot=pd.get_dummies(Toronto_Venues['Venue Category'],prefix='',prefix_sep='')#one hot encoding
#Add Neighborhood and PostalCode column back to Toronto_onehot dataframe
Toronto_onehot['PostalCode']=Toronto_Venues['PostalCode']
columns=[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[:-1])
Toronto_onehot=Toronto_onehot[columns]
Toronto_onehot.head()

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
Toronto_onehot.shape


(1997, 263)

In [64]:
X=Toronto_Venues['Neighborhood']


In [65]:
Toronto_onehot[-1]=X


In [66]:
Toronto_onehot=Toronto_onehot.rename({-1:'Neighborhoods'},axis=1)


In [67]:
Toronto_onehot.head()

,PostalCode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighborhoods
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
3,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village


In [68]:
col=[Toronto_onehot.columns[0]]+[Toronto_onehot.columns[-1]]+list(Toronto_onehot.columns[1:-1])
Toronto_onehot=Toronto_onehot[col]

In [69]:
Toronto_onehot.head()

,PostalCode,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M3A,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,Victoria Village,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's group rows by postal code by taking the mean of the frequency of occurrence of each category¶


In [70]:
Toronto_grouped=Toronto_onehot.groupby(['PostalCode','Neighborhoods'],sort=False).mean().reset_index()

In [71]:
Toronto_grouped.head()

,PostalCode,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,Parkwoods,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,M4A,Victoria Village,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,M5A,Harbourfront,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,M6A,"Lawrence Manor, Lawrence Heights",0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.000000
4,M7A,Queen's Park,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.038462


Now, we print each neighborhood along with top 5 venues for first 10 postal codes

In [72]:
num=5
for code,nbd in zip(Toronto_grouped['PostalCode'][0:10],Toronto_grouped['Neighborhoods'][0:10]):
    print('----'+code+':'+nbd+'----')
    temp=Toronto_grouped[Toronto_grouped['PostalCode']==code].T.reset_index()
    temp=temp.iloc[2:]
    temp.columns=['venue category','freq']
    temp['freq']=temp['freq'].astype(float)
    temp.sort_values(by=['freq'],ascending=False,inplace=True)
    temp=temp.round({'freq':2})
    print(temp.reset_index(drop=True).head(num))

----M3A:Parkwoods----
               venue category  freq
0                        Park  0.25
1        Fast Food Restaurant  0.25
2           Food & Drink Shop  0.25
3  Construction & Landscaping  0.25
4  Modern European Restaurant  0.00
----M4A:Victoria Village----
          venue category  freq
0            Pizza Place  0.25
1           Hockey Arena  0.25
2            Coffee Shop  0.25
3  Portuguese Restaurant  0.25
4    Monument / Landmark  0.00
----M5A:Harbourfront----
  venue category  freq
0    Coffee Shop  0.17
1           Park  0.07
2         Bakery  0.07
3            Pub  0.07
4           Café  0.05
----M6A:Lawrence Manor, Lawrence Heights----
           venue category  freq
0          Clothing Store  0.27
1  Furniture / Home Store  0.18
2       Accessories Store  0.09
3     Arts & Crafts Store  0.09
4                Boutique  0.09
----M7A:Queen's Park----
     venue category  freq
0       Coffee Shop  0.23
1  Sushi Restaurant  0.12
2     Burrito Place  0.08
3       Yoga Studi

In [73]:
def most_common_venues(row,num_of_values):
    row_sorted=row.iloc[2:].sort_values(ascending=False)
    return row_sorted.index.values[0:num_of_values]

We now create a dataframe to contain top 10 venues of each postal code
We first create the name of columns of such dataframe

In [74]:

num_of_values=10
indicators=['st','nd','rd']
columns=['PostalCode','Neighborhoods']
for ind in np.arange(num_of_values):
    try:
        columns.append('{}{} Most common venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most common venue'.format(ind+1))

In [75]:

Toronto_top=pd.DataFrame(columns=columns)

In [76]:
Toronto_top['PostalCode']=Toronto_grouped['PostalCode']
Toronto_top['Neighborhoods']=Toronto_grouped['Neighborhoods']

In [77]:

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_top.iloc[ind][2:]=most_common_venues(Toronto_grouped.iloc[ind],num_of_values)

In [78]:
Toronto_top.head()

,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,M3A,Parkwoods,Food & Drink Shop,Fast Food Restaurant,Park,Construction & Landscaping,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,M4A,Victoria Village,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Harbourfront,Coffee Shop,Pub,Park,Bakery,Café,Farmers Market,Asian Restaurant,French Restaurant,Bank,Spa
3,M6A,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store,Coffee Shop,Electronics Store,Eastern European Restaurant
4,M7A,Queen's Park,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,College Auditorium,Persian Restaurant,Café,Diner,Salad Place,Sandwich Place


Now we run K means clustering to cluster the similar postal codes/Neighborhoods¶


In [79]:
Toronto_grouped.drop(['PostalCode','Neighborhoods'],axis=1,inplace=True)
Toronto_grouped.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.038462


In [80]:
k=5 # number of clusters
k_means=KMeans(n_clusters=k,random_state=0)
k_means.fit(Toronto_grouped)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [81]:

k_means.labels_[0:10] 

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 2])

In [82]:
Toronto_top.insert(0,' Cluster labels',k_means.labels_)

In [83]:

Toronto_top.head()

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,2,M3A,Parkwoods,Food & Drink Shop,Fast Food Restaurant,Park,Construction & Landscaping,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,0,M4A,Victoria Village,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,0,M5A,Harbourfront,Coffee Shop,Pub,Park,Bakery,Café,Farmers Market,Asian Restaurant,French Restaurant,Bank,Spa
3,0,M6A,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store,Coffee Shop,Electronics Store,Eastern European Restaurant
4,0,M7A,Queen's Park,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,College Auditorium,Persian Restaurant,Café,Diner,Salad Place,Sandwich Place


In [84]:
Toronto_data=df
Toronto_top=Toronto_top.join(Toronto_data.set_index('PostalCode'),on=['PostalCode'])

In [85]:

Toronto_top.head()

,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,Parkwoods,Food & Drink Shop,Fast Food Restaurant,Park,Construction & Landscaping,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,North York,Parkwoods,43.753259,-79.329656
1,0,M4A,Victoria Village,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,North York,Victoria Village,43.725882,-79.315572
2,0,M5A,Harbourfront,Coffee Shop,Pub,Park,Bakery,Café,Farmers Market,Asian Restaurant,French Restaurant,Bank,Spa,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,0,M6A,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store,Coffee Shop,Electronics Store,Eastern European Restaurant,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,0,M7A,Queen's Park,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,College Auditorium,Persian Restaurant,Café,Diner,Salad Place,Sandwich Place,Queen's Park,Queen's Park,43.662301,-79.389494


In [86]:
Toronto_top.drop(['Neighborhood'],axis=1,inplace=True)


In [87]:
Toronto_top.head()


,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue,Borough,Latitude,Longitude
0,2,M3A,Parkwoods,Food & Drink Shop,Fast Food Restaurant,Park,Construction & Landscaping,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,North York,43.753259,-79.329656
1,0,M4A,Victoria Village,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,North York,43.725882,-79.315572
2,0,M5A,Harbourfront,Coffee Shop,Pub,Park,Bakery,Café,Farmers Market,Asian Restaurant,French Restaurant,Bank,Spa,Downtown Toronto,43.654260,-79.360636
3,0,M6A,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store,Coffee Shop,Electronics Store,Eastern European Restaurant,North York,43.718518,-79.464763
4,0,M7A,Queen's Park,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,College Auditorium,Persian Restaurant,Café,Diner,Salad Place,Sandwich Place,Queen's Park,43.662301,-79.389494


Visualising the clusters

In [88]:
map_clusters=folium.Map([latitude,longitude],zoom_start=10)#Creating the map


In [89]:
x=np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [90]:
for code,nbd,lat,lng,clus in zip(Toronto_top['PostalCode'],Toronto_top['Neighborhoods'],Toronto_top['Latitude'],Toronto_top['Longitude'],Toronto_top[' Cluster labels']):
    label=folium.Popup('Postal Code:{};Neighborhoods:{};Cluster:{}'.format(code,nbd,clus),parse_html=True)
    folium.CircleMarker([lat,lng],popup=label,color=rainbow[clus-1],
                       fill=True,
                       fill_color=rainbow[clus-1],
                       fill_opacity=0.6,
                       radius=5).add_to(map_clusters)

In [91]:
map_clusters

Examining the clusters
Cluster 1: Cluster Label 0

In [92]:
Toronto_top.loc[Toronto_top[' Cluster labels']==0,Toronto_top.columns[:-3]].reset_index(drop=True)


,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,0,M4A,Victoria Village,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,0,M5A,Harbourfront,Coffee Shop,Pub,Park,Bakery,Café,Farmers Market,Asian Restaurant,French Restaurant,Bank,Spa
2,0,M6A,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Arts & Crafts Store,Miscellaneous Shop,Boutique,Accessories Store,Coffee Shop,Electronics Store,Eastern European Restaurant
3,0,M7A,Queen's Park,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,College Auditorium,Persian Restaurant,Café,Diner,Salad Place,Sandwich Place
4,0,M1B,"Malvern, Rouge",Fast Food Restaurant,Farmers Market,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0,M5X,"First Canadian Place, Underground city",Coffee Shop,Café,Sandwich Place,Hotel,Bank,Japanese Restaurant,Gym,Deli / Bodega,Restaurant,Sushi Restaurant
75,0,M8X,"The Kingsway, Montgomery Road, Old Mill North",River,Pool,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
76,0,M4Y,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Restaurant,Coffee Shop,Gay Bar,Burrito Place,Indian Restaurant,Pizza Place,Mediterranean Restaurant,Fast Food Restaurant
77,0,M7Y,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Park,Garden Center,Skate Park,Brewery,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Butcher


We observe that Cluster 1 has neighborhoods with Playground, Yoga Studio, Drugstore, Dim Sum Restaurant, Diner, Discount Store, Dog Run, Doner Restaurant, Donut Shp in Top 10 venues with the two restaurants share similar ranking
Cluster 2: Cluster Label 1

In [93]:
Toronto_top.loc[Toronto_top[' Cluster labels']==1,Toronto_top.columns[:-3]].reset_index(drop=True)


,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,1,M3M,Downsview Central,Baseball Field,Food Truck,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Farmers Market
1,1,M9M,"Emery, Humberlea",Construction & Landscaping,Baseball Field,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dessert Shop
2,1,M8Y,"Kingsway Park South East, Mimico NE, Sunnylea,...",Baseball Field,Locksmith,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore


Cluster 3: Cluster Label 2

In [94]:
Toronto_top.loc[Toronto_top[' Cluster labels']==2,Toronto_top.columns[:-3]].reset_index(drop=True)


,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,2,M3A,Parkwoods,Food & Drink Shop,Fast Food Restaurant,Park,Construction & Landscaping,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,2,M6B,Glencairn,Pizza Place,Park,Bakery,Italian Restaurant,Colombian Restaurant,Comfort Food Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
2,2,M4C,Woodbine Heights,Beer Store,Athletics & Sports,Park,Skating Rink,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
3,2,M6C,Humewood-Cedarvale,Field,Hockey Arena,Home Service,Park,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store
4,2,M6E,Caledonia-Fairbanks,Park,Women's Store,Pool,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
5,2,M4J,East Toronto,Intersection,Park,Bus Stop,Convenience Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
6,2,M3K,"CFB Toronto, Downsview East",Park,Airport,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
7,2,M6L,"Upwood Park, Downsview, North Park",Basketball Court,Bakery,Park,Construction & Landscaping,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
8,2,M2M,"Willowdale, Newtonbrook",Piano Bar,Gym,Park,Electronics Store,Eastern European Restaurant,Escape Room,Drugstore,Donut Shop,Deli / Bodega,Doner Restaurant
9,2,M4N,Lawrence Park,Bus Line,Business Service,Swim School,Park,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio


We observe that most of the neighborhoods in the Cluster have Park, Yoga Studio, Doner Restaurant, Donut Shop, Discount Store, Dog Run in top 10 venues
Cluster 4: Cluster label 3

In [95]:
Toronto_top.loc[Toronto_top[' Cluster labels']==3,Toronto_top.columns[:-3]].reset_index(drop=True)


,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,3,M9B,"West Deane Park, Cloverdale, Princess Gardens,...",Home Service,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
1,3,M1C,"Port Union, Highland Creek, Rouge Hill",Home Service,Bar,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


We observe that the neighborhoods have same top 10 venues with same ranking
Cluster 5: Cluster label 4

In [96]:
Toronto_top.loc[Toronto_top[' Cluster labels']==4,Toronto_top.columns[:-3]].reset_index(drop=True)


,Cluster labels,PostalCode,Neighborhoods,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,4,M1J,Scarborough Village,Playground,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop


We observe that maximum number of neighborhoods are in Cluster 2: Cluster label 1¶
